In [3]:
from utils import get_clues, generate_abs_path
from src.opensource.llama_frontend import chat_pipeline
from langchain.llms import HuggingFacePipeline
import pandas as pd
from src.opensource.llama_backend import generate_text
import os
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from src.opensource.templates import SYS_FALCON_TEMPLATE, INST_FALCON_TEMPLATE, SYS_LLAMA_TEMPLATE, INST_LLAMA_TEMPLATE
from const import STATE_CLUES_NOTES_DICT
from dotenv import load_dotenv
from langchain import HuggingFaceHub

In [4]:
load_dotenv()

True

In [5]:
conversation_buffer = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

In [6]:
inst_template = PromptTemplate.from_template(INST_FALCON_TEMPLATE)
inst_template = inst_template.format(state='Assam')
print(inst_template)

Name the object on the basis of the above clues from Assam. After your guess, I will let you know if you are correct or not. I do not need to know your reasoning behind the answer. Just tell me the answer and nothing else. If you do not know the answer, say that you do not know the answer. Format your answer in the form of ANSWER: your_answer_here where your_answer_here is your guess. Do not deviate from this answer format.


In [ ]:
# from utils.utils import state_clue_notes_dict


In [7]:
df = pd.read_csv('/home/t-sahuja/cultural_artifacts/clues/assam/artifacts_clues.csv')

In [55]:
arti = df.iloc[8]['artifact']
clues = df.iloc[8]['clues'].strip().split('\n')

In [56]:
print(clues)
print(arti)

['The book that studies the life of a very important religious guru.', 'It is not a book used for chanting']
Chitra-Bhagavata


In [57]:
inst_template = PromptTemplate.from_template(INST_FALCON_TEMPLATE)
inst_template = inst_template.format(state='Assam')

In [58]:
output = ""
for j, clue in enumerate(clues):
    output += f"CLUE-{j+1}: {clue.strip()}\n"
fin_clues = output.strip()

In [59]:
fin_clues

'CLUE-1: The book that studies the life of a very important religious guru.\nCLUE-2: It is not a book used for chanting'

In [13]:
llm = HuggingFacePipeline(pipeline=generate_text("tiiuae/falcon-7b-instruct"))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
 llm = HuggingFaceHub(
        huggingfacehub_api_token=os.environ["HF_TOKEN"],
        repo_id="tiiuae/falcon-7b-instruct",
        model_kwargs={"temperature": 0.1, "max_new_tokens": 500, "do_sample":False},
    )

In [68]:
conversation_buffer = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

In [69]:
 agent = chat_pipeline(
            clue_list=fin_clues,
            prompt_text=SYS_FALCON_TEMPLATE,
            conversation_buffer=conversation_buffer,
            llm=llm,
        )

In [70]:
gg = agent.predict(human_input=inst_template)

In [71]:
print(gg)

 The object you are thinking of is a <b>Tulkus</b>. It is a Tibetan Buddhist religious figure that is believed to have attained enlightenment and is considered to be a manifestation of a deity. It is not a book used for chanting.


In [72]:
conversation_buffer

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Name the object on the basis of the above clues from Assam. After your guess, I will let you know if you are correct or not. I do not need to know your reasoning behind the answer. Just tell me the answer and nothing else. If you do not know the answer, say that you do not know the answer. Format your answer in the form of ANSWER: your_answer_here where your_answer_here is your guess. Do not deviate from this answer format.', additional_kwargs={}, example=False), AIMessage(content=' The object you are thinking of is a <b>Tulkus</b>. It is a Tibetan Buddhist religious figure that is believed to have attained enlightenment and is considered to be a manifestation of a deity. It is not a book used for chanting.', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='chat_history', k=2)

In [73]:
guess2 = agent.predict(human_input="Your first guess is not correct. While making your second guess, please stick to the format as ANSWER: your_answer_here")

In [74]:
guess2

' The object you are thinking of is a <b>Tulkus</b>. It is a Tibetan Buddhist religious figure that is believed to have attained enlightenment and is considered to be a manifestation of a deity. It is not a book used for chanting.\nUser '

In [27]:
conversation_buffer.clear()

In [ ]:
df_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])

In [ ]:
def get_outputs(df, df_eval):
    for i, row in df.iterrows():
        print(f"artifact--{i}---")
        clues = row["clues"].strip().split("\n")
        artifact = row["artifact"].lower().strip()
        output = ""
        for j, clue in enumerate(clues):
            output += f"CLUE-{j+1}: {clue.strip()}\n"
            fin_clues = output.strip()

        agent = chat_pipeline(
            clue_list=fin_clues,
            prompt_text=SYS_FALCON_TEMPLATE,
            conversation_buffer=conversation_buffer,
            model_id="meta-llama/Llama-2-13b-chat-hf"
        )
        guess1 = agent.predict(human_input=inst_template)
        print(guess1, "chek")
        guess1 = guess1.split(":")[1].strip().lower() if len(guess1.split(":")) > 1 else guess1.strip()
        if artifact in guess1:
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": "NA",
                    "ground_truth": artifact,
                    "clues": row["clues"].strip(),
                },
                ignore_index=True,
            )
            conversation_buffer.clear()
            continue
        else:
            guess2 = agent.predict(human_input="Your first guess is not correct. While making your second guess, please stick to the format as ANSWER: your_answer_here")
            guess2 = guess2.split(":")[1].strip().lower() if len(guess2.split(':')) > 1 else guess2.strip()
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": guess2,
                    "ground_truth": artifact,
                    "clues": row["clues"].strip(),
                },
                ignore_index=True,
            )
        conversation_buffer.clear()
    return df_eval

In [ ]:
conversation_buffer.clear()

In [ ]:
df_eval = get_outputs(df, df_eval)

In [ ]:
gg = generate_abs_path('results/opensource/llama')
os.path.exists(gg)

In [ ]:
df_eval.to_csv('jharkhand_evals_with_state_name.csv', index=False)

In [ ]:
def compile_results(STATE_CLUES_NOTES_DICT, output_dir):
    for key, val in STATE_CLUES_NOTES_DICT.items():
        curr_path = os.path.join(output_dir,key)
        if not os.path.exists(curr_path):
            os.makedirs(curr_path)
        
        clue_path = val[0]
        notes_path = val[1] if len(val) > 1 else None
        
#         print("Getting results for {key}ate")
        print(f"getting results for {key} state")
        conversation_buffer.clear()
        df_clues_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])
        df_clues = pd.read_csv(clue_path)
        
        
        print(f"Running clues eval for {key} state")
        clues_result_path = os.path.join(curr_path,'eval_clues.csv')
        if not os.path.exists(clues_result_path):
            df_clues_eval = get_outputs(df_clues, df_clues_eval)
            df_clues_eval.to_csv(clues_result_path, index=False)
        else:
            print(f"Clue eval results already exist for {key} state")
        
        if notes_path:
            conversation_buffer.clear()
            df_notes = pd.read_csv(notes_path)
            df_notes_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])
            notes_result_path = os.path.join(curr_path,'eval_notes.csv')
            if not os.path.exists(notes_result_path):
                print(f"Running notes eval for {key} state")
                df_notes_eval = get_outputs(df_notes, df_clues_eval)
                df_notes_eval.to_csv(notes_result_path, index=False)
            else:
                print(f"Notes eval results already exist for {key} state")
                
            

In [ ]:
compile_results(STATE_CLUES_NOTES_DICT, '/home/t-sahuja/cultural_artifacts/results/opensource/llama')